In [1]:
import cx_Oracle
from IPython.display import IFrame    #to display pdf file
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import pickle
from py2casefold import casefold
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# SQL0

In [64]:
#Access to Oracle DB
host = 'tncluster6.cbi.net'
port = 1521
s_name = 'EDW1TS_EX.cbi.net'
dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
db_ts = cx_Oracle.connect('amartinezcotto', 'Kohlia#743', dsn_tns)
cursor = db_ts.cursor()

query = '''select /*+ parallel(6) */ 
 a14.TDLINX_STORE_CD  RTL_TDLINX_STORE_CD,
 a14.STORE_CD  RTL_STORE_CD,
 max(a14.STORE_STTS_DSC)  RTL_STORE_STTS_DSC,
 a14.STATE_CD  RTL_STATE_CD,
 max(a14.FIPS_COUNTY_DSC)  RTL_FIPS_COUNTY_DSC,
 a14.CITY_DSC  RTL_CITY_DSC,
 a14.ADDR_01_DSC  RTL_ADDR_01_DSC,
 a14.PREMISE_TYPE_CD  RTL_PREMISE_TYPE_CD,
 max(a14.CHANNEL_DSC)  RTL_CHANNEL_DSC,
 a14.SUBCHANNEL_DSC  RTL_SUBCHANNEL_DSC,
 sum(a11.EQV_QTY)  WJXBFS1
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_L_DT_XFORM_MTD_VW a12
   on  (a11.DATE_ID = a12.MTD_DT_ID)
 join EDW.BI_CRN_D_DISTRIBUTOR_VW a13
   on  (a11.DIST_ID = a13.DIST_ID)
 join EDW.BI_CRN_D_RETAILER_VW a14
   on  (a11.RETAILER_ID = a14.RETAILER_ID)
 join EDW.BI_D_DATE_VW a15
   on  (a12.DATE_ID = a15.DATE_ID)
where (a15.CBBD_BUSI_DAYS_NUM = 2
 and a13.STATE_PROVINCE_COUNTRY_CD in ('162'))
group by a14.TDLINX_STORE_CD, 
 a14.STORE_CD,
 a14.SAME_STORE_RPT_DT_ID,
 a14.SUBCHANNEL_CD,
 a14.SUBCHANNEL_DSC,
 a14.STORE_STTS_CD,
 a14.STATE_CD,
 a14.FIPS_COUNTY_CD,
 a14.STATE_CD,
 a14.CITY_DSC,
 a14.AREA_CD,
 a14.ADDR_01_DSC,
 a13.STATE_PROVINCE_COUNTRY_CD,
 a14.PREMISE_TYPE_CD,
 a14.NO_OF_STORES_CD,
 a13.SALES_GRP_CD,
 a14.CHANNEL_CD 
order by a14.TDLINX_STORE_CD'''



In [65]:
try:
    cursor.execute(query)
    names = [ x[0] for x in cursor.description]
    rows = cursor.fetchall()
    sql_df = pd.DataFrame(rows, columns=names)

finally:
    if cursor is not None:
        cursor.close()


# SQL1 

In [61]:
#Store impact1
host = 'tncluster6.cbi.net'
port = 1521
s_name = 'EDW1TS_EX.cbi.net'
dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
db_ts = cx_Oracle.connect('amartinezcotto', 'Kohlia#743', dsn_tns)
cursor = db_ts.cursor()

query = '''select /*+ parallel(6) */  a13.TDLINX_STORE_CD  RTL_TDLINX_STORE_CD,
 a13.STORE_CD  RTL_STORE_CD,
 a13.SUBCHANNEL_DSC  RTL_SUBCHANNEL_DSC,
 max(a13.STORE_STTS_DSC)  RTL_STORE_STTS_DSC,
 a13.CITY_DSC  RTL_CITY_DSC,
 a13.ADDR_01_DSC  RTL_ADDR_01_DSC,
 a13.PREMISE_TYPE_CD  RTL_PREMISE_TYPE_CD,
 max(a13.CHANNEL_DSC)  RTL_CHANNEL_DSC,
 sum(a11.EQV_QTY)  WJXBFS1
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_L_DT_XFORM_CY_VW a12
   on  (a11.DATE_ID = a12.CY_DT_ID)
 join EDW.BI_CRN_D_RETAILER_VW a13
   on  (a11.RETAILER_ID = a13.RETAILER_ID)
 join EDW.BI_CRN_D_DISTRIBUTOR_VW a14
   on  (a11.DIST_ID = a14.DIST_ID)
 join EDW.BI_D_DATE_VW a15
   on  (a12.DATE_ID = a15.DATE_ID)
where (a15.CBBD_BUSI_DAYS_NUM = 2
 and a14.STATE_PROVINCE_COUNTRY_CD in ('162'))
group by  a13.TDLINX_STORE_CD,
 a13.STORE_CD,
 a13.SUBCHANNEL_CD,
 a13.SUBCHANNEL_DSC,
 a13.STORE_STTS_CD,
 a13.CITY_DSC,
 a13.ADDR_01_DSC,
 a13.PREMISE_TYPE_CD,
 a13.CHANNEL_CD  
 order by TDLINX_STORE_CD'''

In [62]:
try:
    cursor.execute(query)
    names = [ x[0] for x in cursor.description]
    rows = cursor.fetchall()
    sql_df1 = pd.DataFrame(rows, columns=names)

finally:
    if cursor is not None:
        cursor.close()
            


In [73]:
#check which ones does sql lack
sql_df1['RTL_TDLINX_STORE_CD'].describe()

count        2748
unique       2723
top       7396948
freq            2
Name: RTL_TDLINX_STORE_CD, dtype: object

In [143]:
#find the duplicate RTL_TDLINX_STORE_CD in sql_df1
last1 = '0'
duplicate_index_1 = []
for index in range(len(sql_df1)):

    if sql_df1.loc[index, 'RTL_TDLINX_STORE_CD'] == last1:
        duplicate_index_1.append(index - 1)
        duplicate_index_1.append(index)
    last1 = sql_df1.loc[index, 'RTL_TDLINX_STORE_CD']
print 'there are', len(duplicate_index_1), 'duplicated rows in sql_df1'

there are 50 duplicated rows in sql_df1


In [144]:
#show the duplicate rows  
result_df1 = pd.DataFrame()
for eachIndex in duplicate_index_1:
    tempDF = sql_df1.loc[eachIndex, ].to_frame().transpose()
    frames = [result_df1, tempDF]
    result_df1 = pd.concat(frames)
result_df1.head()

,RTL_TDLINX_STORE_CD,RTL_STORE_CD,RTL_SUBCHANNEL_DSC,RTL_STORE_STTS_DSC,RTL_CITY_DSC,RTL_ADDR_01_DSC,RTL_PREMISE_TYPE_CD,RTL_CHANNEL_DSC,WJXBFS1
84,0091635,200091635,SUPERMARKET-CONVENTIONAL,CLOSED STORE,ABILENE,1900 N BUCKEYE AVE,OFF,GROCERY,6
85,0091635,300091635,SUPERMARKET-CONVENTIONAL,OPEN STORE,ABILENE,1900 N BUCKEYE AVE,OFF,GROCERY,9
94,0091808,100091808,CONVENTIONAL CONVENIENCE,CLOSED STORE,HUTCHINSON,428 E 4TH AVE,OFF,CONVENIENCE STORE,2
95,0091808,200091808,CONVENTIONAL CONVENIENCE,OPEN STORE,HUTCHINSON,428 E 4TH AVE,OFF,CONVENIENCE STORE,3
97,0091819,100091819,CONVENTIONAL CONVENIENCE,CLOSED STORE,HUTCHINSON,901 E 11TH AVE,OFF,CONVENIENCE STORE,2


# SQL2

In [102]:
#StroreImpact1 sql2
#Store impact1
host = 'tncluster6.cbi.net'
port = 1521
s_name = 'EDW1TS_EX.cbi.net'
dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
db_ts = cx_Oracle.connect('amartinezcotto', 'Kohlia#743', dsn_tns)
cursor = db_ts.cursor()

query =  '''select /*+ parallel(6) */  a14.TDLINX_STORE_CD  RTL_TDLINX_STORE_CD,
 a14.STORE_CD  RTL_STORE_CD,
 a14.SUBCHANNEL_DSC  RTL_SUBCHANNEL_DSC,
 max(a14.STORE_STTS_DSC)  RTL_STORE_STTS_DSC,
 a14.CITY_DSC  RTL_CITY_DSC,
 a14.ADDR_01_DSC  RTL_ADDR_01_DSC,
 max(a14.PREMISE_TYPE_DSC)  RTL_PREMISE_TYPE_DSC,
 max(a14.CHANNEL_DSC)  RTL_CHANNEL_DSC,
 sum(a11.EQV_QTY)  WJXBFS1
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_L_DT_XFORM_CY_VW a12
   on  (a11.DATE_ID = a12.CY_DT_ID)
 join EDW.BI_D_DATE_VW a13
   on  (a12.DATE_ID = a13.LAST_2YR_DT_ID)
 join EDW.BI_CRN_D_RETAILER_VW a14
   on  (a11.RETAILER_ID = a14.RETAILER_ID)
 join EDW.BI_CRN_D_DISTRIBUTOR_VW a15
   on  (a11.DIST_ID = a15.DIST_ID)
 join EDW.BI_D_DATE_VW a16
   on  (a13.DATE_ID = a16.DATE_ID)
where (a16.CBBD_BUSI_DAYS_NUM = 2
 and a15.STATE_PROVINCE_COUNTRY_CD in ('162'))
group by  a14.TDLINX_STORE_CD,
 a14.STORE_CD,
 a14.SUBCHANNEL_CD,
 a14.SUBCHANNEL_DSC,
 a14.STORE_STTS_CD,
 a14.CITY_DSC,
 a14.ADDR_01_DSC,
 a14.PREMISE_TYPE_CD,
 a14.CHANNEL_CD 
order by a14.TDLINX_STORE_CD'''

try:
    cursor.execute(query)
    names = [ x[0] for x in cursor.description]
    rows = cursor.fetchall()
    sql_df2 = pd.DataFrame(rows, columns=names)

finally:
    if cursor is not None:
        cursor.close()
            

In [104]:
#check the duplicate stoer TDLINX
sql_df2['RTL_TDLINX_STORE_CD'].describe()


count        2853
unique       2742
top       0961650
freq            4
Name: RTL_TDLINX_STORE_CD, dtype: object

In [136]:
#find the duplicate RTL_TDLINX_STORE_CD in sql_df2
last_2 = '0'
duplicate_index_2 = []
for index in range(len(sql_df2)):
    if sql_df2.loc[index, 'RTL_TDLINX_STORE_CD'] == last_2:
        duplicate_index_2.append(index-1)
        duplicate_index_2.append(index)
    last_2 = sql_df2.loc[index, 'RTL_TDLINX_STORE_CD']
print 'there are', len(duplicate_index_2), 'duplicated rows in sql_df2'

there are 222 duplicated rows in sql_df2


In [139]:
#show the duplicate rows  
result_df2 = pd.DataFrame()
for eachIndex in duplicate_index_2:
    tempDF = sql_df2.loc[eachIndex, ].to_frame().transpose()
    frames = [result_df2, tempDF]
    result_df2 = pd.concat(frames)
result_df2.head()

,RTL_TDLINX_STORE_CD,RTL_STORE_CD,RTL_SUBCHANNEL_DSC,RTL_STORE_STTS_DSC,RTL_CITY_DSC,RTL_ADDR_01_DSC,RTL_PREMISE_TYPE_DSC,RTL_CHANNEL_DSC,WJXBFS1
31,0090793,100090793,SUPERMARKET-CONVENTIONAL,CLOSED STORE,SENECA,506 NORTH ST US HWY 36,OFF PREMISE,GROCERY,1
32,0090793,200090793,SUPERMARKET-CONVENTIONAL,CLOSED STORE,SENECA,506 N ST US HWY 36,OFF PREMISE,GROCERY,2
43,0090962,100090962,SUPERMARKET-CONVENTIONAL,CLOSED STORE,TOPEKA,2620 NE SARDOU AVE,OFF PREMISE,GROCERY,1
44,0090962,200090962,SUPERMARKET-CONVENTIONAL,CLOSED STORE,TOPEKA,2620 NE SARDOU AVE,OFF PREMISE,GROCERY,6
44,0090962,200090962,SUPERMARKET-CONVENTIONAL,CLOSED STORE,TOPEKA,2620 NE SARDOU AVE,OFF PREMISE,GROCERY,6


# SQL3

In [147]:
#StroreImpact1 sql3

host = 'tncluster6.cbi.net'
port = 1521
s_name = 'EDW1TS_EX.cbi.net'
dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
db_ts = cx_Oracle.connect('amartinezcotto', 'Kohlia#743', dsn_tns)
cursor = db_ts.cursor()

query =  '''select /*+ parallel(6) */ a14.TDLINX_STORE_CD  RTL_TDLINX_STORE_CD,
 a14.STORE_CD  RTL_STORE_CD,
 a14.SUBCHANNEL_DSC  RTL_SUBCHANNEL_DSC,
 max(a14.STORE_STTS_DSC)  RTL_STORE_STTS_DSC,
 a14.CITY_DSC  RTL_CITY_DSC,
 a14.ADDR_01_DSC  RTL_ADDR_01_DSC,
 max(a14.PREMISE_TYPE_DSC)  RTL_PREMISE_TYPE_DSC,
 max(a14.CHANNEL_DSC)  RTL_CHANNEL_DSC,
 sum(a11.EQV_QTY)  WJXBFS1
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_L_DT_XFORM_CY_VW a12
   on  (a11.DATE_ID = a12.CY_DT_ID)
 join EDW.BI_D_DATE_VW a13
   on  (a12.DATE_ID = a13.LAST_YR_DT_ID)
 join EDW.BI_CRN_D_RETAILER_VW a14
   on  (a11.RETAILER_ID = a14.RETAILER_ID)
 join EDW.BI_CRN_D_DISTRIBUTOR_VW a15
   on  (a11.DIST_ID = a15.DIST_ID)
 join EDW.BI_D_DATE_VW a16
   on  (a13.DATE_ID = a16.DATE_ID)
where (a16.CBBD_BUSI_DAYS_NUM = 2
 and a15.STATE_PROVINCE_COUNTRY_CD in ('162'))
group by  a14.TDLINX_STORE_CD,
 a14.STORE_CD,
 a14.SUBCHANNEL_CD,
 a14.SUBCHANNEL_DSC,
 a14.STORE_STTS_CD,
 a14.CITY_DSC,
 a14.ADDR_01_DSC,
 a14.PREMISE_TYPE_CD,
 a14.CHANNEL_CD 
order by a14.TDLINX_STORE_CD'''

try:
    cursor.execute(query)
    names = [ x[0] for x in cursor.description]
    rows = cursor.fetchall()
    sql_df3 = pd.DataFrame(rows, columns=names)

finally:
    if cursor is not None:
        cursor.close()
            

In [148]:
#check the duplicate stoer TDLINX
sql_df3['RTL_TDLINX_STORE_CD'].describe()

count        2935
unique       2811
top       1804005
freq            3
Name: RTL_TDLINX_STORE_CD, dtype: object

In [149]:
#find the duplicate RTL_TDLINX_STORE_CD in sql_df2
last_3 = '0'
duplicate_index_3 = []
for index in range(len(sql_df3)):
    if sql_df3.loc[index, 'RTL_TDLINX_STORE_CD'] == last_3:
        duplicate_index_3.append(index-1)
        duplicate_index_3.append(index)
    last_3 = sql_df3.loc[index, 'RTL_TDLINX_STORE_CD']
print 'there are', len(duplicate_index_3), 'duplicated rows in sql_df3'

there are 248 duplicated rows in sql_df3


In [150]:
#show the duplicate rows  
result_df3 = pd.DataFrame()
for eachIndex in duplicate_index_3:
    tempDF = sql_df3.loc[eachIndex, ].to_frame().transpose()
    frames = [result_df3, tempDF]
    result_df3 = pd.concat(frames)
result_df3.head()

,RTL_TDLINX_STORE_CD,RTL_STORE_CD,RTL_SUBCHANNEL_DSC,RTL_STORE_STTS_DSC,RTL_CITY_DSC,RTL_ADDR_01_DSC,RTL_PREMISE_TYPE_DSC,RTL_CHANNEL_DSC,WJXBFS1
20,0090570,100090570,CONVENTIONAL CONVENIENCE,CLOSED STORE,MISSION,7400 SHAWNEE MISSION PKWY,OFF PREMISE,CONVENIENCE STORE,248.5
21,0090570,200090570,CONVENTIONAL CONVENIENCE,OPEN STORE,MISSION,7400 SHAWNEE MISSION PKWY,OFF PREMISE,CONVENIENCE STORE,175.166
79,0091561,100091561,SUPERMARKET-CONVENTIONAL,CLOSED STORE,INDEPENDENCE,416 W MYRTLE ST,OFF PREMISE,GROCERY,2
80,0091561,200091561,SUPERMARKET-CONVENTIONAL,OPEN STORE,INDEPENDENCE,416 W MYRTLE ST,OFF PREMISE,GROCERY,6
127,0092052,100092052,CONVENTIONAL CONVENIENCE,CLOSED STORE,HUGOTON,1000 S MAIN ST,OFF PREMISE,CONVENIENCE STORE,39


In [ ]:
#find the duplicate RTL_TDLINX_STORE_CD in sql_df2
last_3 = '0'
duplicate_index_3 = []
for index in range(len(sql_df3)):
    if sql_df3.loc[index, 'RTL_TDLINX_STORE_CD'] == last_3:
        duplicate_index_3.append(index-1)
        duplicate_index_3.append(index)
    last_3 = sql_df3.loc[index, 'RTL_TDLINX_STORE_CD']
print 'there are', len(duplicate_index_3), 'duplicated rows in sql_df3'

#show the duplicate rows  
result_df3 = pd.DataFrame()
for eachIndex in duplicate_index_3:
    tempDF = sql_df3.loc[eachIndex, ].to_frame().transpose()
    frames = [result_df3, tempDF]
    result_df3 = pd.concat(frames)
result_df3.head()

In [164]:
duplicate_store_TDLINX_1 = list(result_df1['RTL_TDLINX_STORE_CD'])
duplicate_store_TDLINX_2 = list(result_df2['RTL_TDLINX_STORE_CD'])
duplicate_store_TDLINX_3 = list(result_df3['RTL_TDLINX_STORE_CD'])

In [169]:
#get the unique stores from those three results.

,RTL_TDLINX_STORE_CD,RTL_STORE_CD,RTL_SUBCHANNEL_DSC,RTL_STORE_STTS_DSC,RTL_CITY_DSC,RTL_ADDR_01_DSC,RTL_PREMISE_TYPE_CD,RTL_CHANNEL_DSC,WJXBFS1
84,0091635,200091635,SUPERMARKET-CONVENTIONAL,CLOSED STORE,ABILENE,1900 N BUCKEYE AVE,OFF,GROCERY,6
85,0091635,300091635,SUPERMARKET-CONVENTIONAL,OPEN STORE,ABILENE,1900 N BUCKEYE AVE,OFF,GROCERY,9


In [180]:
one_list = duplicate_store_TDLINX_1 + duplicate_store_TDLINX_2 + duplicate_store_TDLINX_3

In [187]:
no_duplicate = set(one_list)
no_duplicate = list(no_duplicate)
print 'there are', len(no_duplicate), 'that made change to their stores.'

there are 240 that made change to their stores.


In [193]:
#create a dataframe that contains all the changes for that 240 stores
final_df = pd.DataFrame()
for each in no_duplicate:
    if each in duplicate_store_TDLINX_3:
        tempDF = result_df3.loc[result_df3['RTL_TDLINX_STORE_CD'] == each]
    elif each in duplicate_store_TDLINX_2:
        tempDF = result_df2.loc[result_df2['RTL_TDLINX_STORE_CD'] == each]
    else:
        tempDF = result_df1.loc[result_df1['RTL_TDLINX_STORE_CD'] == each]
    frames = [final_df, tempDF]
    final_df = pd.concat(frames)
final_df.head()

,RTL_ADDR_01_DSC,RTL_CHANNEL_DSC,RTL_CITY_DSC,RTL_PREMISE_TYPE_CD,RTL_PREMISE_TYPE_DSC,RTL_STORE_CD,RTL_STORE_STTS_DSC,RTL_SUBCHANNEL_DSC,RTL_TDLINX_STORE_CD,WJXBFS1
2841,210 N WASHINGTON ST,DINING,WICHITA,NaN,ON PREMISE,107419910,CLOSED STORE,CASUAL DINING,7419910,5.25
2842,210 N WASHINGTON ST,DINING,WICHITA,NaN,ON PREMISE,207419910,OPEN STORE,CASUAL DINING,7419910,6.5
1422,9705 ROSEHILL RD,LIQUOR,LENEXA,NaN,OFF PREMISE,201606170,CLOSED STORE,CONVENTIONAL LIQUOR,1606170,1112.67
1423,9705 ROSEHILL RD,LIQUOR,LENEXA,NaN,OFF PREMISE,301606170,OPEN STORE,CONVENTIONAL LIQUOR,1606170,260.417
212,101 S 4TH ST,CONVENIENCE STORE,LEOTI,NaN,OFF PREMISE,100282555,CLOSED STORE,CONVENTIONAL CONVENIENCE,0282555,52


In [ ]:
#test for quantity

In [217]:
#StroreImpact1 sql3

host = 'tncluster6.cbi.net'
port = 1521
s_name = 'EDW1TS_EX.cbi.net'
dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
db_ts = cx_Oracle.connect('amartinezcotto', 'Kohlia#743', dsn_tns)
cursor = db_ts.cursor()

query =  '''select /*+ parallel(6) */ a14.TDLINX_STORE_CD  RTL_TDLINX_STORE_CD,
 a14.STORE_CD  RTL_STORE_CD,
 a14.SUBCHANNEL_DSC  RTL_SUBCHANNEL_DSC,
 max(a14.STORE_STTS_DSC)  RTL_STORE_STTS_DSC,
 a14.CITY_DSC  RTL_CITY_DSC,
 a14.ADDR_01_DSC  RTL_ADDR_01_DSC,
 max(a14.PREMISE_TYPE_DSC)  RTL_PREMISE_TYPE_DSC,
 max(a14.CHANNEL_DSC)  RTL_CHANNEL_DSC,
 avg(a11.EQV_QTY) QTY
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_L_DT_XFORM_CY_VW a12
   on  (a11.DATE_ID = a12.CY_DT_ID)
 join EDW.BI_D_DATE_VW a13
   on  (a12.DATE_ID = a13.LAST_YR_DT_ID)
 join EDW.BI_CRN_D_RETAILER_VW a14
   on  (a11.RETAILER_ID = a14.RETAILER_ID)
 join EDW.BI_CRN_D_DISTRIBUTOR_VW a15
   on  (a11.DIST_ID = a15.DIST_ID)
 join EDW.BI_D_DATE_VW a16
   on  (a13.DATE_ID = a16.DATE_ID)
where (a16.CBBD_BUSI_DAYS_NUM = 2
 and a15.STATE_PROVINCE_COUNTRY_CD in ('162'))
group by  a14.TDLINX_STORE_CD,
 a14.STORE_CD,
 a14.SUBCHANNEL_CD,
 a14.SUBCHANNEL_DSC,
 a14.STORE_STTS_CD,
 a14.CITY_DSC,
 a14.ADDR_01_DSC,
 a14.PREMISE_TYPE_CD,
 a14.CHANNEL_CD 
order by a14.TDLINX_STORE_CD'''

try:
    cursor.execute(query)
    names = [ x[0] for x in cursor.description]
    rows = cursor.fetchall()
    tempDF = pd.DataFrame(rows, columns=names)

finally:
    if cursor is not None:
        cursor.close()
            

In [ ]:
#StroreImpact1 sql3

host = 'tncluster6.cbi.net'
port = 1521
s_name = 'EDW1TS_EX.cbi.net'
dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
db_ts = cx_Oracle.connect('amartinezcotto', 'Kohlia#743', dsn_tns)
cursor = db_ts.cursor()

query =  '''select /*+ parallel(6) */ a14.TDLINX_STORE_CD  RTL_TDLINX_STORE_CD,
 a14.STORE_CD  RTL_STORE_CD,
 a14.SUBCHANNEL_DSC  RTL_SUBCHANNEL_DSC,
 max(a14.STORE_STTS_DSC)  RTL_STORE_STTS_DSC,
 a14.CITY_DSC  RTL_CITY_DSC,
 a14.ADDR_01_DSC  RTL_ADDR_01_DSC,
 max(a14.PREMISE_TYPE_DSC)  RTL_PREMISE_TYPE_DSC,
 max(a14.CHANNEL_DSC)  RTL_CHANNEL_DSC,
 avg(a11.EQV_QTY) QTY
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_L_DT_XFORM_CY_VW a12
   on  (a11.DATE_ID = a12.CY_DT_ID)
 join EDW.BI_D_DATE_VW a13
   on  (a12.DATE_ID = a13.LAST_YR_DT_ID)
 join EDW.BI_CRN_D_RETAILER_VW a14
   on  (a11.RETAILER_ID = a14.RETAILER_ID)
 join EDW.BI_CRN_D_DISTRIBUTOR_VW a15
   on  (a11.DIST_ID = a15.DIST_ID)
 join EDW.BI_D_DATE_VW a16
   on  (a13.DATE_ID = a16.DATE_ID)
where (a16.CBBD_BUSI_DAYS_NUM = 2
 and a15.STATE_PROVINCE_COUNTRY_CD in ('162'))
group by  a14.TDLINX_STORE_CD,
 a14.STORE_CD,
 a14.SUBCHANNEL_CD,
 a14.SUBCHANNEL_DSC,
 a14.STORE_STTS_CD,
 a14.CITY_DSC,
 a14.ADDR_01_DSC,
 a14.PREMISE_TYPE_CD,
 a14.CHANNEL_CD 
order by a14.TDLINX_STORE_CD'''

try:
    cursor.execute(query)
    names = [ x[0] for x in cursor.description]
    rows = cursor.fetchall()
    tempDF = pd.DataFrame(rows, columns=names)

finally:
    if cursor is not None:
        cursor.close()
            

In [218]:
tempDF.head()

,RTL_TDLINX_STORE_CD,RTL_STORE_CD,RTL_SUBCHANNEL_DSC,RTL_STORE_STTS_DSC,RTL_CITY_DSC,RTL_ADDR_01_DSC,RTL_PREMISE_TYPE_DSC,RTL_CHANNEL_DSC,QTY
0,0090293,100090293,SUPERMARKET-CONVENTIONAL,OPEN STORE,BONNER SPRINGS,501 COMMERCIAL DR,OFF PREMISE,GROCERY,1.309091
1,0090308,100090308,SUPERETTE,OPEN STORE,LA CYGNE,403 E MARKET ST,OFF PREMISE,GROCERY,0.555556
2,0090317,200090317,SUPERMARKET-CONVENTIONAL,OPEN STORE,LAWRENCE,1740 MASSACHUSETTS AVE,OFF PREMISE,GROCERY,1.539326
3,0090323,100090323,SUPERMARKET-CONVENTIONAL,OPEN STORE,LAWRENCE,2300 LOUISIANA ST,OFF PREMISE,GROCERY,1.874993
4,0090339,100090339,CONVENTIONAL CONVENIENCE,OPEN STORE,LEAVENWORTH,1031 METROPOLITAN ST,OFF PREMISE,CONVENIENCE STORE,1.000000


# Project starts here

# Step1: Get Full calendar year data for each year, and store them in three dataframes

In [85]:
#2YL

#StroreImpact1 sql3

host = 'tncluster6.cbi.net'
port = 1521
s_name = 'EDW1TS_EX.cbi.net'
dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
db_ts = cx_Oracle.connect('amartinezcotto', 'Kohlia#743', dsn_tns)
cursor = db_ts.cursor()

query =  '''select /*+ parallel(6) */ a14.TDLINX_STORE_CD  RTL_TDLINX_STORE_CD,
 a14.STORE_CD  RTL_STORE_CD,
 max(a14.CHANNEL_DSC)  RTL_CHANNEL_DSC,
 a14.SUBCHANNEL_DSC  RTL_SUBCHANNEL_DSC,
 max(a14.STORE_STTS_DSC)  RTL_STORE_STTS_DSC,
 max(a14.PREMISE_TYPE_DSC)  RTL_PREMISE_TYPE_DSC,
 a14.ADDR_01_DSC  RTL_ADDR_01_DSC,
 sum(a11.EQV_QTY)  EQV_QTY_YA2
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_L_DT_XFORM_CY_VW a12
   on  (a11.DATE_ID = a12.CY_DT_ID)
 join EDW.BI_D_DATE_VW a13
   on  (a12.DATE_ID = a13.LAST_2YR_DT_ID)
 join EDW.BI_CRN_D_RETAILER_VW a14
   on  (a11.RETAILER_ID = a14.RETAILER_ID)
 join EDW.BI_CRN_D_DISTRIBUTOR_VW a15
   on  (a11.DIST_ID = a15.DIST_ID)
 join EDW.BI_D_DATE_VW a16
   on  (a13.DATE_ID = a16.DATE_ID)
where (a16.CBBD_BUSI_DAYS_NUM = 2
 and a15.STATE_PROVINCE_COUNTRY_CD in ('162'))
group by a14.TDLINX_STORE_CD,
 a14.STORE_CD,
 a14.SUBCHANNEL_CD,
 a14.SUBCHANNEL_DSC,
 a14.STORE_STTS_CD,
 a14.ADDR_01_DSC,
 a14.PREMISE_TYPE_CD,
 a14.CHANNEL_CD 
order by a14.TDLINX_STORE_CD
'''

try:
    cursor.execute(query)
    names = [ x[0] for x in cursor.description]
    rows = cursor.fetchall()
    YA2_DF = pd.DataFrame(rows, columns=names)

finally:
    if cursor is not None:
        cursor.close()

YA2_DF.head()

,RTL_TDLINX_STORE_CD,RTL_STORE_CD,RTL_CHANNEL_DSC,RTL_SUBCHANNEL_DSC,RTL_STORE_STTS_DSC,RTL_PREMISE_TYPE_DSC,RTL_ADDR_01_DSC,EQV_QTY_YA2
0,0090292,100090292,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,112 OAK ST,106.4999
1,0090293,100090293,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,501 COMMERCIAL DR,53.0000
2,0090308,100090308,GROCERY,SUPERETTE,OPEN STORE,OFF PREMISE,403 E MARKET ST,10.0000
3,0090317,200090317,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,1740 MASSACHUSETTS AVE,51.0000
4,0090323,100090323,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,2300 LOUISIANA ST,23.0000


In [86]:
#YA
host = 'tncluster6.cbi.net'
port = 1521
s_name = 'EDW1TS_EX.cbi.net'
dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
db_ts = cx_Oracle.connect('amartinezcotto', 'Kohlia#743', dsn_tns)
cursor = db_ts.cursor()

query =  ''' select /*+ parallel(6) */ a14.TDLINX_STORE_CD  RTL_TDLINX_STORE_CD,
 a14.STORE_CD  RTL_STORE_CD,
 max(a14.CHANNEL_DSC)  RTL_CHANNEL_DSC,
 a14.SUBCHANNEL_DSC  RTL_SUBCHANNEL_DSC,
 max(a14.STORE_STTS_DSC)  RTL_STORE_STTS_DSC,
 max(a14.PREMISE_TYPE_DSC)  RTL_PREMISE_TYPE_DSC,
 a14.ADDR_01_DSC  RTL_ADDR_01_DSC,
 sum(a11.EQV_QTY)  EQV_QTY_YA
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_L_DT_XFORM_CY_VW a12
   on  (a11.DATE_ID = a12.CY_DT_ID)
 join EDW.BI_D_DATE_VW a13
   on  (a12.DATE_ID = a13.LAST_YR_DT_ID)
 join EDW.BI_CRN_D_RETAILER_VW a14
   on  (a11.RETAILER_ID = a14.RETAILER_ID)
 join EDW.BI_CRN_D_DISTRIBUTOR_VW a15
   on  (a11.DIST_ID = a15.DIST_ID)
 join EDW.BI_D_DATE_VW a16
   on  (a13.DATE_ID = a16.DATE_ID)
where (a16.CBBD_BUSI_DAYS_NUM = 2
 and a15.STATE_PROVINCE_COUNTRY_CD in ('162'))
group by a14.TDLINX_STORE_CD,
 a14.STORE_CD,
 a14.SUBCHANNEL_CD,
 a14.SUBCHANNEL_DSC,
 a14.STORE_STTS_CD,
 a14.ADDR_01_DSC,
 a14.PREMISE_TYPE_CD,
 a14.CHANNEL_CD  
order by a14.TDLINX_STORE_CD'''

try:
    cursor.execute(query)
    names = [ x[0] for x in cursor.description]
    rows = cursor.fetchall()
    YA_DF = pd.DataFrame(rows, columns=names)

finally:
    if cursor is not None:
        cursor.close()

YA_DF.head()

,RTL_TDLINX_STORE_CD,RTL_STORE_CD,RTL_CHANNEL_DSC,RTL_SUBCHANNEL_DSC,RTL_STORE_STTS_DSC,RTL_PREMISE_TYPE_DSC,RTL_ADDR_01_DSC,EQV_QTY_YA
0,0090293,100090293,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,501 COMMERCIAL DR,72.0000
1,0090308,100090308,GROCERY,SUPERETTE,OPEN STORE,OFF PREMISE,403 E MARKET ST,5.0000
2,0090317,200090317,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,1740 MASSACHUSETTS AVE,137.0000
3,0090323,100090323,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,2300 LOUISIANA ST,194.9993
4,0090339,100090339,CONVENIENCE STORE,CONVENTIONAL CONVENIENCE,OPEN STORE,OFF PREMISE,1031 METROPOLITAN ST,8.0000


In [87]:
#TY

host = 'tncluster6.cbi.net'
port = 1521
s_name = 'EDW1TS_EX.cbi.net'
dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
db_ts = cx_Oracle.connect('amartinezcotto', 'Kohlia#743', dsn_tns)
cursor = db_ts.cursor()

query =  ''' select /*+ parallel(6) */  a13.TDLINX_STORE_CD  RTL_TDLINX_STORE_CD,
 a13.STORE_CD  RTL_STORE_CD,
 max(a13.CHANNEL_DSC)  RTL_CHANNEL_DSC,   
 a13.SUBCHANNEL_DSC  RTL_SUBCHANNEL_DSC,
 max(a13.STORE_STTS_DSC)  RTL_STORE_STTS_DSC,
 max(a13.PREMISE_TYPE_DSC)  RTL_PREMISE_TYPE_DSC,
 a13.ADDR_01_DSC  RTL_ADDR_01_DSC,
 sum(a11.EQV_QTY)  EQV_QTY_TY
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_L_DT_XFORM_CY_VW a12
   on  (a11.DATE_ID = a12.CY_DT_ID)
 join EDW.BI_CRN_D_RETAILER_VW a13
   on  (a11.RETAILER_ID = a13.RETAILER_ID)
 join EDW.BI_CRN_D_DISTRIBUTOR_VW a14
   on  (a11.DIST_ID = a14.DIST_ID)
 join EDW.BI_D_DATE_VW a15
   on  (a12.DATE_ID = a15.DATE_ID)
where (a15.CBBD_BUSI_DAYS_NUM = 2
 and a14.STATE_PROVINCE_COUNTRY_CD in ('162'))
group by  a13.TDLINX_STORE_CD,
 a13.STORE_CD,
 a13.SUBCHANNEL_CD,
 a13.SUBCHANNEL_DSC,
 a13.STORE_STTS_CD,
 a13.ADDR_01_DSC,
 a13.PREMISE_TYPE_CD,
 a13.CHANNEL_CD 
order by  a13.TDLINX_STORE_CD
 '''

try:
    cursor.execute(query)
    names = [ x[0] for x in cursor.description]
    rows = cursor.fetchall()
    TY_DF = pd.DataFrame(rows, columns=names)

finally:
    if cursor is not None:
        cursor.close()



TY_DF.head()

,RTL_TDLINX_STORE_CD,RTL_STORE_CD,RTL_CHANNEL_DSC,RTL_SUBCHANNEL_DSC,RTL_STORE_STTS_DSC,RTL_PREMISE_TYPE_DSC,RTL_ADDR_01_DSC,EQV_QTY_TY
0,0090293,100090293,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,501 COMMERCIAL DR,31.0000
1,0090308,100090308,GROCERY,SUPERETTE,OPEN STORE,OFF PREMISE,403 E MARKET ST,1.0000
2,0090317,200090317,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,1740 MASSACHUSETTS AVE,66.0000
3,0090323,100090323,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,2300 LOUISIANA ST,118.4998
4,0090339,100090339,CONVENIENCE STORE,CONVENTIONAL CONVENIENCE,OPEN STORE,OFF PREMISE,1031 METROPOLITAN ST,1.0000


# Step 2: Merge the three dataframes into one

In [94]:
#create a combined_DF to store all three years data
combined_DF = pd.concat([TY_DF, YA_DF, YA2_DF], ignore_index=True)
#change the order of columns
cols = ['RTL_TDLINX_STORE_CD', 'RTL_STORE_CD', 'RTL_CHANNEL_DSC', 'RTL_SUBCHANNEL_DSC', 'RTL_STORE_STTS_DSC', 'RTL_PREMISE_TYPE_DSC', 'RTL_ADDR_01_DSC', 'EQV_QTY_TY','EQV_QTY_YA', 'EQV_QTY_YA2']
combined_DF = combined_DF[cols]
#sort the dataframe by RTL_TDLINX_STORE_CD
combined_DF = combined_DF.sort_values('RTL_TDLINX_STORE_CD', ascending = True)
#reset the index 
combined_DF.reset_index(level=None, drop=True, inplace=True)
no_duplicate_df = combined_DF[['RTL_TDLINX_STORE_CD', 'RTL_STORE_CD', 'RTL_CHANNEL_DSC', 'RTL_SUBCHANNEL_DSC', 'RTL_STORE_STTS_DSC', 'RTL_PREMISE_TYPE_DSC', 'RTL_ADDR_01_DSC']]
no_duplicate_df = no_duplicate_df.drop_duplicates()
combined_DF = no_duplicate_df.merge(TY_DF, on=['RTL_TDLINX_STORE_CD', 'RTL_STORE_CD', 'RTL_CHANNEL_DSC', 'RTL_SUBCHANNEL_DSC', 'RTL_STORE_STTS_DSC', 'RTL_PREMISE_TYPE_DSC', 'RTL_ADDR_01_DSC'], how='left')
combined_DF = combined_DF.merge(YA_DF, on=['RTL_TDLINX_STORE_CD', 'RTL_STORE_CD', 'RTL_CHANNEL_DSC', 'RTL_SUBCHANNEL_DSC', 'RTL_STORE_STTS_DSC', 'RTL_PREMISE_TYPE_DSC', 'RTL_ADDR_01_DSC'], how='left')
combined_DF = combined_DF.merge(YA2_DF, on=['RTL_TDLINX_STORE_CD', 'RTL_STORE_CD', 'RTL_CHANNEL_DSC', 'RTL_SUBCHANNEL_DSC', 'RTL_STORE_STTS_DSC', 'RTL_PREMISE_TYPE_DSC', 'RTL_ADDR_01_DSC'], how='left')

combined_DF.head()

,RTL_TDLINX_STORE_CD,RTL_STORE_CD,RTL_CHANNEL_DSC,RTL_SUBCHANNEL_DSC,RTL_STORE_STTS_DSC,RTL_PREMISE_TYPE_DSC,RTL_ADDR_01_DSC,EQV_QTY_TY,EQV_QTY_YA,EQV_QTY_YA2
0,0090292,100090292,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,112 OAK ST,NaN,NaN,106.4999
1,0090293,100090293,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,501 COMMERCIAL DR,31.0000,72.0000,53.0000
2,0090308,100090308,GROCERY,SUPERETTE,OPEN STORE,OFF PREMISE,403 E MARKET ST,1.0000,5.0000,10.0000
3,0090317,200090317,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,1740 MASSACHUSETTS AVE,66.0000,137.0000,51.0000
4,0090323,100090323,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,2300 LOUISIANA ST,118.4998,194.9993,23.0000


# Step3: Subset the stores that had closed and re-opened

In [189]:
previous_store_num = '0'
subset_DF = pd.DataFrame()
for index in range(len(combined_DF)):
    if combined_DF.loc[index, 'RTL_TDLINX_STORE_CD'] == previous_store_num:
        tempDF1 = combined_DF.loc[index-1, :].to_frame().transpose()
        tempDF2 = combined_DF.loc[index, :].to_frame().transpose()
        frames = [subset_DF, tempDF1, tempDF2]
        subset_DF = pd.concat(frames)
    previous_store_num = combined_DF.loc[index, 'RTL_TDLINX_STORE_CD']

#remove the duplicated rows
subset_DF = subset_DF.drop_duplicates()
subset_DF = subset_DF.sort_values(['RTL_TDLINX_STORE_CD', 'RTL_STORE_CD'], ascending = True)
subset_DF.head()

,RTL_TDLINX_STORE_CD,RTL_STORE_CD,RTL_CHANNEL_DSC,RTL_SUBCHANNEL_DSC,RTL_STORE_STTS_DSC,RTL_PREMISE_TYPE_DSC,RTL_ADDR_01_DSC,EQV_QTY_TY,EQV_QTY_YA,EQV_QTY_YA2
8,0090383,200090383,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,701 6TH ST,NaN,NaN,18
9,0090383,300090383,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,701 6TH ST,NaN,6,NaN
23,0090570,100090570,CONVENIENCE STORE,CONVENTIONAL CONVENIENCE,CLOSED STORE,OFF PREMISE,7400 SHAWNEE MISSION PKWY,NaN,248.5,345.75
22,0090570,200090570,CONVENIENCE STORE,CONVENTIONAL CONVENIENCE,OPEN STORE,OFF PREMISE,7400 SHAWNEE MISSION PKWY,236.083,175.166,NaN
38,0090793,100090793,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,506 NORTH ST US HWY 36,NaN,NaN,1


# Step4: Get the BEERTYPE for each store, each year

In [190]:
#TY
host = 'tncluster6.cbi.net'
port = 1521
s_name = 'EDW1TS_EX.cbi.net'
dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
db_ts = cx_Oracle.connect('amartinezcotto', 'Kohlia#743', dsn_tns)
cursor = db_ts.cursor()

query =  ''' select /*+ parallel(6) */ a14.STORE_CD  RTL_STORE_CD,
 'NonLowPoint' as "TY_BEERTYPE"
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_CRN_D_ITEM_VW a12
   on  (a11.ITEM_ID = a12.ITEM_ID)
 join EDW.BI_CRN_D_REL_TIME_DETAIL_VW a13
   on  (a11.DATE_ID = a13.REL_DATE_ID)
 join EDW.BI_CRN_D_RETAILER_VW a14
   on  (a11.RETAILER_ID = a14.RETAILER_ID)
where (a13.REL_TIME_CD in ('L3_TY')
 and a14.STATE_CD in ('KS')
 and a12.MASTER_SKU_CD not in ('80013450', '80013452', '80013455', '80013979', '80013456', '80013475', '80061325', '80013477', '80013982', '80013980', '80059842', '80058839', '80014020', '80014022', '80014023', '80014024', '80014025', '80013992', '80013991', '80013464', '80013466', '80013469', '80013471', '80013472', '80013473', '80013478', '80031998', '80013485', '80056926', '80058838', '80019746', '80031994', '80015966', '80014017', '80012704', '80014015', '80014009', '80014011', '80024732', '80013994', '80056172', '80014010'))
group by a14.STORE_CD
 '''

try:
    cursor.execute(query)
    names = [ x[0] for x in cursor.description]
    rows = cursor.fetchall()
    TY_NonLowPoint_DF = pd.DataFrame(rows, columns=names)

finally:
    if cursor is not None:
        cursor.close()



TY_NonLowPoint_DF.head()



,RTL_STORE_CD,TY_BEERTYPE
0,104010171,NonLowPoint
1,102194529,NonLowPoint
2,201914562,NonLowPoint
3,101734453,NonLowPoint
4,101403794,NonLowPoint


In [191]:
#TY
host = 'tncluster6.cbi.net'
port = 1521
s_name = 'EDW1TS_EX.cbi.net'
dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
db_ts = cx_Oracle.connect('amartinezcotto', 'Kohlia#743', dsn_tns)
cursor = db_ts.cursor()

query =  ''' select /*+ parallel(6) */ a14.STORE_CD  RTL_STORE_CD,
 'LowPoint' as "TY_BEERTYPE"
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_CRN_D_ITEM_VW a12
   on  (a11.ITEM_ID = a12.ITEM_ID)
 join EDW.BI_CRN_D_REL_TIME_DETAIL_VW a13
   on  (a11.DATE_ID = a13.REL_DATE_ID)
 join EDW.BI_CRN_D_RETAILER_VW a14
   on  (a11.RETAILER_ID = a14.RETAILER_ID)
where (a13.REL_TIME_CD in ('L3_TY')
 and a14.STATE_CD in ('KS')
 and  a14.STORE_CD not in  
(select /*+ parallel(6) */ a14.STORE_CD  RTL_STORE_CD
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_CRN_D_ITEM_VW a12
   on  (a11.ITEM_ID = a12.ITEM_ID)
 join EDW.BI_CRN_D_REL_TIME_DETAIL_VW a13
   on  (a11.DATE_ID = a13.REL_DATE_ID)
 join EDW.BI_CRN_D_RETAILER_VW a14
   on  (a11.RETAILER_ID = a14.RETAILER_ID)
where (a13.REL_TIME_CD in ('L3_TY')
 and a14.STATE_CD in ('KS')
 and a12.MASTER_SKU_CD not in ('80013450', '80013452', '80013455', '80013979', '80013456', '80013475', '80061325', '80013477', '80013982', '80013980', '80059842', '80058839', '80014020', '80014022', '80014023', '80014024', '80014025', '80013992', '80013991', '80013464', '80013466', '80013469', '80013471', '80013472', '80013473', '80013478', '80031998', '80013485', '80056926', '80058838', '80019746', '80031994', '80015966', '80014017', '80012704', '80014015', '80014009', '80014011', '80024732', '80013994', '80056172', '80014010'))
 group by a14.STORE_CD))
group by a14.STORE_CD
 '''


try:
    cursor.execute(query)
    names = [ x[0] for x in cursor.description]
    rows = cursor.fetchall()
    TY_LowPoint_DF = pd.DataFrame(rows, columns=names)

finally:
    if cursor is not None:
        cursor.close()



TY_LowPoint_DF.head()

,RTL_STORE_CD,TY_BEERTYPE
0,100090584,LowPoint
1,201725558,LowPoint
2,100696211,LowPoint
3,100682170,LowPoint
4,100090531,LowPoint


In [192]:
#make sure that no store belongs to more than one type
TY_LowPoint_DF.merge(TY_NonLowPoint_DF, on = 'RTL_STORE_CD', how = 'inner')

,RTL_STORE_CD,TY_BEERTYPE_x,TY_BEERTYPE_y


In [193]:
TY_beerType_DF = pd.concat([TY_NonLowPoint_DF, TY_LowPoint_DF])
TY_beerType_DF.head()

,RTL_STORE_CD,TY_BEERTYPE
0,104010171,NonLowPoint
1,102194529,NonLowPoint
2,201914562,NonLowPoint
3,101734453,NonLowPoint
4,101403794,NonLowPoint


In [194]:
#merge with subset_DF 
subset_DF = pd.merge(subset_DF, TY_beerType_DF, on="RTL_STORE_CD", how="left")
subset_DF

,RTL_TDLINX_STORE_CD,RTL_STORE_CD,RTL_CHANNEL_DSC,RTL_SUBCHANNEL_DSC,RTL_STORE_STTS_DSC,RTL_PREMISE_TYPE_DSC,RTL_ADDR_01_DSC,EQV_QTY_TY,EQV_QTY_YA,EQV_QTY_YA2,TY_BEERTYPE
0,0090383,200090383,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,701 6TH ST,NaN,NaN,18,NaN
1,0090383,300090383,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,701 6TH ST,NaN,6,NaN,NaN
2,0090570,100090570,CONVENIENCE STORE,CONVENTIONAL CONVENIENCE,CLOSED STORE,OFF PREMISE,7400 SHAWNEE MISSION PKWY,NaN,248.5,345.75,NaN
3,0090570,200090570,CONVENIENCE STORE,CONVENTIONAL CONVENIENCE,OPEN STORE,OFF PREMISE,7400 SHAWNEE MISSION PKWY,236.083,175.166,NaN,LowPoint
4,0090793,100090793,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,506 NORTH ST US HWY 36,NaN,NaN,1,NaN
5,0090793,200090793,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,506 N ST US HWY 36,NaN,3,2,NaN
6,0090962,100090962,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,2620 NE SARDOU AVE,NaN,NaN,1,NaN
7,0090962,200090962,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,2620 NE SARDOU AVE,NaN,NaN,6,NaN
8,0090962,300090962,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,2620 NE SARDOU AVE,5,27,5,LowPoint
9,0091124,100091124,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,2311 M ST,NaN,15,10,NaN


In [195]:
#YA_NonLowPoint_DF

host = 'tncluster6.cbi.net'
port = 1521
s_name = 'EDW1TS_EX.cbi.net'
dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
db_ts = cx_Oracle.connect('amartinezcotto', 'Kohlia#743', dsn_tns)
cursor = db_ts.cursor()

query =  ''' select /*+ parallel(6) */ a15.STORE_CD  RTL_STORE_CD,
  'NonLowPoint' as "YA_BEERTYPE"
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_L_DT_XFORM_RANGE_LINEAR_VW a12
   on  (a11.DATE_ID = a12.LAST_YR_DT_ID)
 join EDW.BI_CRN_D_ITEM_VW a13
   on  (a11.ITEM_ID = a13.ITEM_ID)
 join EDW.BI_CRN_D_REL_TIME_DETAIL_VW a14
   on  (a12.DATE_ID = a14.REL_DATE_ID)
 join EDW.BI_CRN_D_RETAILER_VW a15
   on  (a11.RETAILER_ID = a15.RETAILER_ID)
where (a14.REL_TIME_CD in ('L3_TY')
 and a13.MASTER_SKU_CD not in ('80013450', '80013452', '80013455', '80013979', '80013456', '80013475', '80061325', '80013477', '80013982', '80013980', '80059842', '80058839', '80014020', '80014022', '80014023', '80014024', '80014025', '80013992', '80013991', '80013464', '80013466', '80013469', '80013471', '80013472', '80013473', '80013478', '80031998', '80013485', '80056926', '80058838', '80019746', '80031994', '80015966', '80014017', '80012704', '80014015', '80014009', '80014011', '80024732', '80013994', '80056172', '80014010')
 and a15.STATE_CD in ('KS'))
group by a15.STORE_CD
 '''

try:
    cursor.execute(query)
    names = [ x[0] for x in cursor.description]
    rows = cursor.fetchall()
    YA_NonLowPoint_DF = pd.DataFrame(rows, columns=names)

finally:
    if cursor is not None:
        cursor.close()

YA_NonLowPoint_DF.head()

,RTL_STORE_CD,YA_BEERTYPE
0,101000545,NonLowPoint
1,103707016,NonLowPoint
2,101790623,NonLowPoint
3,101403987,NonLowPoint
4,102242163,NonLowPoint


In [196]:
#YA_LowPoint_DF

host = 'tncluster6.cbi.net'
port = 1521
s_name = 'EDW1TS_EX.cbi.net'
dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
db_ts = cx_Oracle.connect('amartinezcotto', 'Kohlia#743', dsn_tns)
cursor = db_ts.cursor()

query =  '''select /*+ parallel(6) */ a15.STORE_CD  RTL_STORE_CD,
  'LowPoint' as "YA_BEERTYPE"
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_L_DT_XFORM_RANGE_LINEAR_VW a12
   on  (a11.DATE_ID = a12.LAST_YR_DT_ID)
 join EDW.BI_CRN_D_ITEM_VW a13
   on  (a11.ITEM_ID = a13.ITEM_ID)
 join EDW.BI_CRN_D_REL_TIME_DETAIL_VW a14
   on  (a12.DATE_ID = a14.REL_DATE_ID)
 join EDW.BI_CRN_D_RETAILER_VW a15
   on  (a11.RETAILER_ID = a15.RETAILER_ID)
where (a14.REL_TIME_CD in ('L3_TY')
 and a15.STATE_CD in ('KS') and 
 a15.STORE_CD  not in  
 (select /*+ parallel(6) */ a15.STORE_CD  RTL_STORE_CD
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_L_DT_XFORM_RANGE_LINEAR_VW a12
   on  (a11.DATE_ID = a12.LAST_YR_DT_ID)
 join EDW.BI_CRN_D_ITEM_VW a13
   on  (a11.ITEM_ID = a13.ITEM_ID)
 join EDW.BI_CRN_D_REL_TIME_DETAIL_VW a14
   on  (a12.DATE_ID = a14.REL_DATE_ID)
 join EDW.BI_CRN_D_RETAILER_VW a15
   on  (a11.RETAILER_ID = a15.RETAILER_ID)
where (a14.REL_TIME_CD in ('L3_TY')
 and a15.STATE_CD in ('KS')
 and a13.MASTER_SKU_CD not in ('80013450', '80013452', '80013455', '80013979', '80013456', '80013475', '80061325', '80013477', '80013982', '80013980', '80059842', '80058839', '80014020', '80014022', '80014023', '80014024', '80014025', '80013992', '80013991', '80013464', '80013466', '80013469', '80013471', '80013472', '80013473', '80013478', '80031998', '80013485', '80056926', '80058838', '80019746', '80031994', '80015966', '80014017', '80012704', '80014015', '80014009', '80014011', '80024732', '80013994', '80056172', '80014010'))
group by a15.STORE_CD))
 group by a15.STORE_CD
 '''


try:
    cursor.execute(query)
    names = [ x[0] for x in cursor.description]
    rows = cursor.fetchall()
    YA_LowPoint_DF = pd.DataFrame(rows, columns=names)

finally:
    if cursor is not None:
        cursor.close()



YA_LowPoint_DF.head()

,RTL_STORE_CD,YA_BEERTYPE
0,101973067,LowPoint
1,100320752,LowPoint
2,101512779,LowPoint
3,100214775,LowPoint
4,105527649,LowPoint


In [197]:
#make sure that no store belongs to more than one type
YA_LowPoint_DF.merge(YA_NonLowPoint_DF, on = 'RTL_STORE_CD')

,RTL_STORE_CD,YA_BEERTYPE_x,YA_BEERTYPE_y


In [198]:
#concat the beerType 
YA_beerType_DF = pd.concat([YA_LowPoint_DF, YA_NonLowPoint_DF])
YA_beerType_DF.head()

,RTL_STORE_CD,YA_BEERTYPE
0,101973067,LowPoint
1,100320752,LowPoint
2,101512779,LowPoint
3,100214775,LowPoint
4,105527649,LowPoint


In [199]:
#merge with subset_DF
subset_DF = pd.merge(subset_DF, YA_beerType_DF, on="RTL_STORE_CD", how="left")
subset_DF.head()

,RTL_TDLINX_STORE_CD,RTL_STORE_CD,RTL_CHANNEL_DSC,RTL_SUBCHANNEL_DSC,RTL_STORE_STTS_DSC,RTL_PREMISE_TYPE_DSC,RTL_ADDR_01_DSC,EQV_QTY_TY,EQV_QTY_YA,EQV_QTY_YA2,TY_BEERTYPE,YA_BEERTYPE
0,0090383,200090383,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,701 6TH ST,NaN,NaN,18,NaN,NaN
1,0090383,300090383,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,701 6TH ST,NaN,6,NaN,NaN,NaN
2,0090570,100090570,CONVENIENCE STORE,CONVENTIONAL CONVENIENCE,CLOSED STORE,OFF PREMISE,7400 SHAWNEE MISSION PKWY,NaN,248.5,345.75,NaN,LowPoint
3,0090570,200090570,CONVENIENCE STORE,CONVENTIONAL CONVENIENCE,OPEN STORE,OFF PREMISE,7400 SHAWNEE MISSION PKWY,236.083,175.166,NaN,LowPoint,NaN
4,0090793,100090793,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,506 NORTH ST US HWY 36,NaN,NaN,1,NaN,NaN


In [200]:
#YA2_NonLowPoint_DF

host = 'tncluster6.cbi.net'
port = 1521
s_name = 'EDW1TS_EX.cbi.net'
dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
db_ts = cx_Oracle.connect('amartinezcotto', 'Kohlia#743', dsn_tns)
cursor = db_ts.cursor()

query =  ''' select /*+ parallel(6) */ a15.STORE_CD  RTL_STORE_CD,
 'NonLowPoint' as "YA2_BEERTYPE"
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_L_DT_XFORM_RANGE_LINEAR_VW a12
   on  (a11.DATE_ID = a12.LAST_2YR_DT_ID)
 join EDW.BI_CRN_D_ITEM_VW a13
   on  (a11.ITEM_ID = a13.ITEM_ID)
 join EDW.BI_CRN_D_REL_TIME_DETAIL_VW a14
   on  (a12.DATE_ID = a14.REL_DATE_ID)
 join EDW.BI_CRN_D_RETAILER_VW a15
   on  (a11.RETAILER_ID = a15.RETAILER_ID)
where (a14.REL_TIME_CD in ('L3_TY')
 and a15.STATE_CD in ('KS')
 and a13.MASTER_SKU_CD not in ('80013450', '80013452', '80013455', '80013979', '80013456', '80013475', '80061325', '80013477', '80013982', '80013980', '80059842', '80058839', '80014020', '80014022', '80014023', '80014024', '80014025', '80013992', '80013991', '80013464', '80013466', '80013469', '80013471', '80013472', '80013473', '80013478', '80031998', '80013485', '80056926', '80058838', '80019746', '80031994', '80015966', '80014017', '80012704', '80014015', '80014009', '80014011', '80024732', '80013994', '80056172', '80014010'))
group by a15.STORE_CD   
 '''

try:
    cursor.execute(query)
    names = [ x[0] for x in cursor.description]
    rows = cursor.fetchall()
    YA2_NonLowPoint_DF = pd.DataFrame(rows, columns=names)

finally:
    if cursor is not None:
        cursor.close()



YA2_NonLowPoint_DF.head()

,RTL_STORE_CD,YA2_BEERTYPE
0,101713286,NonLowPoint
1,100765442,NonLowPoint
2,301403919,NonLowPoint
3,101909409,NonLowPoint
4,101833870,NonLowPoint


In [201]:
#YA2_LowPoint_DF

host = 'tncluster6.cbi.net'
port = 1521
s_name = 'EDW1TS_EX.cbi.net'
dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
db_ts = cx_Oracle.connect('amartinezcotto', 'Kohlia#743', dsn_tns)
cursor = db_ts.cursor()

query =  ''' select /*+ parallel(6) */ a15.STORE_CD  RTL_STORE_CD,
 'LowPoint' as "YA2_BEERTYPE"
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_L_DT_XFORM_RANGE_LINEAR_VW a12
   on  (a11.DATE_ID = a12.LAST_2YR_DT_ID)
 join EDW.BI_CRN_D_ITEM_VW a13
   on  (a11.ITEM_ID = a13.ITEM_ID)
 join EDW.BI_CRN_D_REL_TIME_DETAIL_VW a14
   on  (a12.DATE_ID = a14.REL_DATE_ID)
 join EDW.BI_CRN_D_RETAILER_VW a15
   on  (a11.RETAILER_ID = a15.RETAILER_ID)
where (a14.REL_TIME_CD in ('L3_TY')
 and a15.STATE_CD in ('KS')
 and a15.STORE_CD not in (select /*+ parallel(6) */ a15.STORE_CD  RTL_STORE_CD
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_L_DT_XFORM_RANGE_LINEAR_VW a12
   on  (a11.DATE_ID = a12.LAST_2YR_DT_ID)
 join EDW.BI_CRN_D_ITEM_VW a13
   on  (a11.ITEM_ID = a13.ITEM_ID)
 join EDW.BI_CRN_D_REL_TIME_DETAIL_VW a14
   on  (a12.DATE_ID = a14.REL_DATE_ID)
 join EDW.BI_CRN_D_RETAILER_VW a15
   on  (a11.RETAILER_ID = a15.RETAILER_ID)
where (a14.REL_TIME_CD in ('L3_TY')
 and a15.STATE_CD in ('KS')
 and a13.MASTER_SKU_CD not in ('80013450', '80013452', '80013455', '80013979', '80013456', '80013475', '80061325', '80013477', '80013982', '80013980', '80059842', '80058839', '80014020', '80014022', '80014023', '80014024', '80014025', '80013992', '80013991', '80013464', '80013466', '80013469', '80013471', '80013472', '80013473', '80013478', '80031998', '80013485', '80056926', '80058838', '80019746', '80031994', '80015966', '80014017', '80012704', '80014015', '80014009', '80014011', '80024732', '80013994', '80056172', '80014010'))
group by a15.STORE_CD ))
group by a15.STORE_CD
                            
 '''

try:
    cursor.execute(query)
    names = [ x[0] for x in cursor.description]
    rows = cursor.fetchall()
    YA2_LowPoint_DF = pd.DataFrame(rows, columns=names)

finally:
    if cursor is not None:
        cursor.close()



YA2_LowPoint_DF.head()

,RTL_STORE_CD,YA2_BEERTYPE
0,200090447,LowPoint
1,100431529,LowPoint
2,104038244,LowPoint
3,100916057,LowPoint
4,100505167,LowPoint


In [202]:
YA2_LowPoint_DF.merge(YA2_NonLowPoint_DF, on = 'RTL_STORE_CD')

,RTL_STORE_CD,YA2_BEERTYPE_x,YA2_BEERTYPE_y


In [203]:
#concat
YA2_beerType_DF = pd.concat([YA2_LowPoint_DF, YA2_NonLowPoint_DF])
YA2_beerType_DF.head()

,RTL_STORE_CD,YA2_BEERTYPE
0,200090447,LowPoint
1,100431529,LowPoint
2,104038244,LowPoint
3,100916057,LowPoint
4,100505167,LowPoint


In [204]:
#Merge the subset_DF with beertype
subset_DF = pd.merge(subset_DF, YA2_beerType_DF, on="RTL_STORE_CD", how="left")
subset_DF

,RTL_TDLINX_STORE_CD,RTL_STORE_CD,RTL_CHANNEL_DSC,RTL_SUBCHANNEL_DSC,RTL_STORE_STTS_DSC,RTL_PREMISE_TYPE_DSC,RTL_ADDR_01_DSC,EQV_QTY_TY,EQV_QTY_YA,EQV_QTY_YA2,TY_BEERTYPE,YA_BEERTYPE,YA2_BEERTYPE
0,0090383,200090383,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,701 6TH ST,NaN,NaN,18,NaN,NaN,LowPoint
1,0090383,300090383,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,701 6TH ST,NaN,6,NaN,NaN,NaN,NaN
2,0090570,100090570,CONVENIENCE STORE,CONVENTIONAL CONVENIENCE,CLOSED STORE,OFF PREMISE,7400 SHAWNEE MISSION PKWY,NaN,248.5,345.75,NaN,LowPoint,LowPoint
3,0090570,200090570,CONVENIENCE STORE,CONVENTIONAL CONVENIENCE,OPEN STORE,OFF PREMISE,7400 SHAWNEE MISSION PKWY,236.083,175.166,NaN,LowPoint,NaN,NaN
4,0090793,100090793,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,506 NORTH ST US HWY 36,NaN,NaN,1,NaN,NaN,NaN
5,0090793,200090793,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,506 N ST US HWY 36,NaN,3,2,NaN,LowPoint,NaN
6,0090962,100090962,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,2620 NE SARDOU AVE,NaN,NaN,1,NaN,NaN,NaN
7,0090962,200090962,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,2620 NE SARDOU AVE,NaN,NaN,6,NaN,NaN,LowPoint
8,0090962,300090962,GROCERY,SUPERMARKET-CONVENTIONAL,OPEN STORE,OFF PREMISE,2620 NE SARDOU AVE,5,27,5,LowPoint,LowPoint,NaN
9,0091124,100091124,GROCERY,SUPERMARKET-CONVENTIONAL,CLOSED STORE,OFF PREMISE,2311 M ST,NaN,15,10,NaN,LowPoint,LowPoint
